In [52]:
from langchain_community.chat_models import ChatZhipuAI
from dotenv import load_dotenv
import os

load_dotenv()
key = os.getenv('ZHIPUAI_API_KEY')

## DB类格式

In [53]:
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=key,
    model_name="glm-4",
)

# from langchain_community.utilities import SQLDatabase
# db = SQLDatabase.from_uri("sqlite:///G:/Python Project/llm+sql/ali_langchain.db")
# 
# def get_schema(_):
    # return db.get_table_info()
    # 
# def run_query(query):
    # return db.run(query)
# 
# result = run_query("SELECT COUNT(*) FROM alidata")
# print(result)
# print(db.get_table_info())

## Join表格的实现

In [ ]:
# import pandas as pd
# import sqlite3
# 

# data1 = pd.read_csv('file1.csv')
# 

# data2 = pd.read_csv('file2.csv')
#
# joined_data = pd.merge(data1, data2, on='id')
#
# data_db = "merged_data.db"  # 你希望保存的数据库文件路径
# conn = sqlite3.connect(data_db)
# 
# joined_data.to_sql('merged_table', conn, if_exists='replace', index=False)
# 
# print(f"CSV文件已成功合并并转换为SQLite数据库文件，保存路径为：{data_db}")


## csv格式转db格式

In [42]:
from langchain_community.utilities import SQLDatabase
import pandas as pd
import sqlite3

data = pd.read_csv('imdb_movies2.csv')
data_db = "imdb_movies2.db"  # 更新为你希望保存的数据库文件路径
conn = sqlite3.connect(data_db)

print(f"CSV 文件已成功转换为 SQLite 数据库文件，保存路径为：{data_db}")

table_name = "imdb2"  # 更新为你希望在数据库中使用的表名
data.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

CSV 文件已成功转换为 SQLite 数据库文件，保存路径为：imdb_movies2.db


In [ ]:
db = SQLDatabase.from_uri("sqlite:///G:/Python Project/llm+sql/imdb_movies2.db")

def get_schema(_):
   return db.get_table_info()
  # 
def run_query(query):
   return db.run(query)
# 

result = run_query("SELECT COUNT(*) FROM imdb2")
print(result)
print(db.get_table_info())


In [50]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain.prompts import PromptTemplate

template_sql = (
    "请通过写sql代码来回答对应问题，并且需要基于如下数据库信息：{info}\n"
    "需要回答的问题是：{question}\n"
    "注意仅需要通过sql代码回答，不需要文字\n"
    "代码形式如下：```sql\n...\n```"
)

t_sql=PromptTemplate.from_template(template_sql)

def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_schema,"question":RunnablePassthrough()}
           |t_sql
           |llm
           |StrOutputParser()
           |RunnableLambda(get_sql))
template_sql0 = (
    "请通过综合如下的数据库信息回答问题。问题,sql代码，sql代码的执行结果给出问题的自然语言回答。\n"
    "数据库信息{info}\n"
    "需要回答的问题是：{question}\n"
    "sql代码: {query}\n"
    "sql代码执行结果: {res}"
)

t_sql0=PromptTemplate.from_template(template_sql0)
chain_sql0=({"info":get_schema,"question":RunnablePassthrough(),"query":chain_sql}
            |RunnablePassthrough.assign(res=lambda x: run_query(x["query"]))
            |t_sql0
            |llm
            |StrOutputParser())

In [83]:
question = '找出所有时间最古老电影5名'

In [84]:
chain_sql.invoke(question)

'\nSELECT * \nFROM imdb2 \nORDER BY date_x ASC \nLIMIT 5;\n'

In [85]:
result = chain_sql0.invoke(question)
print(result)

根据提供的数据库信息以及执行的SQL代码结果，以下是时间最古老的5部电影：

1. 《尼罗河的宝石》(The Jewel of the Nile)，上映日期为1986年1月1日。
2. 《星际迷航：起义》(Star Trek: Insurrection)，上映日期为1999年1月1日。
3. 《少年派的奇幻漂流》(Life of Pi)，上映日期为2013年1月1日。
4. 《阿呆与阿瓜2》(Dumb and Dumber To)，上映日期为2015年1月1日。
5. 《疯狂世界》(Rat Race)，上映日期为2002年3月1日。

请注意，这些电影的上映日期是根据提供的SQL查询结果列出的，实际上这些日期可能并不是最准确的发布日期，因为它们显示为"01/01/xxxx"格式，这可能是一个默认或错误的日期输入。然而，根据这个数据，上述列表是按照时间顺序排列的，显示了最古老的电影。
